
SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: MIT

# Traffic Analytics using Multi Class Support in Jetson Platform Services


## Obtain spatio-temporal insights using APIs 

<b>Learning Objective:</b>
- Use APIs to interact with the Analytics Services on Jetson Orin
- Create line-crossing configuration and alerts using VST WebUI
- Generate spatial insights like flow of vehicle movement and heatmaps

<b>This lab is divided into 4 parts: </b>
1. <b>Configure Tripwire (line-crossing) using VST WebUI</b><br>
In the first section, users will experience the VST (Video Storage Toolkit) WebUI to create and configure Tripwire analytics. Not part of the lab, but users can use this WebUI to create virtual boundaries or region of interest (ROI)
2. <b>Vehicle Movement Analytics</b><br>
In this section, users will use the VST (/vst) and Analytics (/emdx) APIs to retrieve time series insights from the Analytics service. They will use this time series data to better understand the flow of vehicle movement. 
3. <b>Heatmap Analytics</b><br>
In this section, user will use the behavior APIs from Analytics service to generate Heatmaps. Heatmaps provide a visualization to understand flow of traffic over time. This is generated by accumulating individual trajectories of movement and mapping it spatially over the region

 

# Vehicle Traffic Movement Analytics and Visualization

### Highway use case
The specific example we will use in this lab includes understanding a 2-lane road scene along with a sidewalk. Each lane has vehicles moving in a specific direction.The concepts in this lab can be used with any streams and any movement of vehicles

### System Concepts
Each passage of interest will be marked using the concept of Tripwire. The actual path a vehicle takes is represented by a Trajectory. The system is capable of identifying the crossing of vehicles across Tripwire and provide a historical count of crossings via an API.

#### Sensor and Streams
The camera used to capture the video of the scene is called as a Sensor. The live feed from the camera is known as a Stream. These are the core concepts of the VST service. All the analytics downstream is also related to Sensors.

#### Tripwire

Tripwire is a construct which is represented as a contigious series of line segments which demarcate a part of the scene across system does counting of crossing. It can easily be thought of as lines marked on the floor of the scene and visualization through a camera live feed.

#### Trajectory
The actual path a vehicle takes is tracked on a frame by frame basis by the system and recorded. Each vehicle is assigned an ID by the system. The total path for a vehicle is represented by the concept of Trajectory in the system. There is an API to retrieve the trajectory of all the vehicles for a given time range. 


# 1. Configure Tripwire using VST Analytics WebUI

The passage of interest will be marked by a Tripwire. For the highway scene we recommend adding a tripwire to represent the lanes i.e South-Bound, North-Bound. 

To identify the distribution of traffic in case of highway scene, we have a tripwire called South-Bound on the right side of the scene, to analyze vehicles going towards South

### Recommended Tripwire configurations

![tripwires](assets/tripwire_config.jpg)

#### API Guide

Analytics microservices API Guide:

https://docs.nvidia.com/moj/emdx_API/index.html

Video Storage Toolkit API Guide:

https://docs.nvidia.com/moj/vst/VST_API_Guide.html



Using the VST LiveStream Analytics feature, configure 1 tripwire as shown recommended above with the same tripwire names as shown in the image. 

Open the VST web UI as opened in lab 1. It can be found at `http://{jetson-device-ip}:30080/vst`

### 1.1 Select the Live stream

Navigate to Live Streams page from left side navigation panel. 
From the Live streams section choose the camera of interest from 'Select sensors'

To zoom the live stream view. Click on the + zoom icon on the right bottom of the 'Live streams' panel.

![VST_Streams.png](assets/VST_1.png)

### 1.2 Open the Analytics panel

Click the down arrow to expand the Analytics panel. 

![VST Analytics Panel](assets/VST_2.png)

### 1.3 Draw Tripwire points with direction arrow

Click on the 'Tripwire' button. 
Using the left mouse button we will click 4 point on the live stream playing video panel. 
Create a tripwire similar to what is shown below by marking out the four points.

![VST_Tripwire_Wire](assets/VST_3.png)

Now mark the entry direction of the tripwire by clicking the two points such that the second point represents the tip of the arrow direction. See image below

![VST_Tripwire_Direction](assets/VST_4.png)

You will be prompted to specify the name of the tripwire. For this particular example specify the name 'South-Bound'. Now click on the 'Done' button

![VST_Save](assets/VST_5.png)

### 1.4 Create remaining tripwires 

Following the steps 1.1-1.3 create tripwire named South-Bound as shown in the recommended image in introduction section

> **Note**: Before continuing, ensure that you have create tripwire in the recommended configuration named "South-Bound"

# 2. Vehicle Movement Analytics: Step by Step Guide

This section includes several Python cells that need to be run in this Notebook. Go through and run the cells in order to learn how to use the APIs to build a heatmap of the vehicle traffic. 

### 2.1 Setup python packages

In [ ]:
!pip install opencv-python==4.9.0.80 requests==2.31.0 numpy==1.26.4 matplotlib==3.6.2 scipy==1.12.0 shapely==2.0.4 pandas==2.2.2

In [ ]:
import cv2, os, requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime, timedelta, timezone
from matplotlib import colors
from scipy.ndimage import gaussian_filter
from urllib import parse

from utils import get_tripwire_name_ID_mapping, get_vst_snapshot, plot_pie_chart, plot_stacked_bar_chart

### 2.2 Configuration 

#### 2.2.1 Define the API endpoints

Make sure the update jetson_device_ip to the IP of your Jetson device below.

In [ ]:
# Analytics microservice endpoint
jetson_device_ip = "172.17.169.139" # TODO: change this to the correct device IP
endpoint = "http://{}:30080/emdx".format(jetson_device_ip)

# VST microservice API endpoint
vst_endpoint = "http://{}:30080/vst".format(jetson_device_ip)

#### 2.2.2 Configure Stream and Sensor

In the VST microservice, each camera is represented as a stream. The stream has a unique ID attribute and a user friendly name.  

<b><i>Note</i></b>: In the Analytics microservices we have a concept of Sensor which is equivalent to the term Stream in VST microservice.

You can get the current sensor information by querying the VST `sensor/list` endpoint as seen below. Additional VST endpoint information can be found in the [MMJ docs](https://docs.nvidia.com/moj/vst/VST_API_Guide.html).

Below, run the sample curl command to fetch all sensors currently available in VST


In [ ]:
!curl http://{jetson_device_ip}:30080/vst/api/v1/sensor/list

Use one of the `name` values from the JSON returned by the curl command above as the value for sensorId below. This sensor will be used for the rest of this lab.

In [ ]:
sensorId = "MOJD2" # TODO: fill in based on one of the "name" values found in the JSON printed above

#### 2.2.3 Object Type

Object type represent the class of vehicles to be configured for Vehicle Movement Analysis. It is a comma separated list of vehicles type. Each vehicle type in the list should be compliant with YOLOv8 COCO dataset class


In [ ]:
objectType = "car,bus,truck"

#### 2.2.4 Get Tripwire Config from API

Following method invokes the Analytics API to retrieve all the tripwire configured for a given sensor
 
Note: All Analytics API endpoint expect Sensor Id as specified in the HTTP parameter sensorId.

Reference: Tripwire configuration API:

https://docs.nvidia.com/moj/emdx/tripwire.html#create-tripwire-configuration

In [ ]:
def get_tripwire_config(endpoint, sensorId):
    """
    Gets the tripwire configurations on a camera sensor

    Keyword arguments:
    endpoint -- the API endpoint of eMDX
    sensorId -- the ID of the camera sensor
    """
    parameters = {}
    parameters['sensorId'] = sensorId
    encoded_parameters = parse.urlencode(parameters)
    url = '{}/api/v2/config/tripwire?{}'.format(endpoint, encoded_parameters)
    r = requests.get(url,verify=False)
    if r.status_code == 200:
        response = r.json()
        return response

You can query this using the cURL command to call analytics microservice `config/tripwire` endpoint to get the current tripwires for a given sensor as seen below. Note the need to specify sensor ID as you set in the previous section.

In [ ]:
!curl http://{jetson_device_ip}:30080/emdx/api/v2/config/tripwire?sensorId={sensorId}

In Analytics microservice each Tripwire is identified by a unique ID and a user friendly name attribute. Update the tripwireNames array below with the name of the four tripwires you created earlier in section 1. These names should also appear in the JSON above returned by the curl command.

In [ ]:
# If the Tripwire names are different than update the 'tripwireNames' list
tripwireNames = ["South-Bound","North-Bound"]

Once we have the tripwire configurations, we will match specified tripwire names with names of the retrieved tripwire configs to determine id of the each specified tripwire.


In [ ]:
tripwire_config = get_tripwire_config(endpoint, sensorId)
tripwireIdMap, tripwireNameMap = get_tripwire_name_ID_mapping(tripwire_config, tripwireNames)
tripwireIds = list(tripwireIdMap.keys())
tripwireId = tripwireIds[0]

In [ ]:
# toTimestamp is the current time while fromTimestamp is the timestamp 5 minutes ago. Change these values as desired.
toTimestamp = datetime.utcnow()
fromTimestamp = toTimestamp - timedelta(minutes=5)

toTimestamp = toTimestamp.isoformat("T")[0:-3] + "Z"
fromTimestamp = fromTimestamp.isoformat("T")[0:-3] + "Z"

# Timerange for which analytics needs to be fetched, in iso8601 formats in milliseconds

In [ ]:
# the duration b/w toTimestamp and fromTimestamp is divided into fixed size windows
# update the desired size in seconds
windowSize = 60

# 2.3 Vehicle Movement Analytics 

#### 2.3.1 Retrieve the Tripwire entry / exits histogram for a given time range from API

This function calls Tripwire Analytics Histogram API to get entry and exit counts for a given timerange and bucketed into fixed intervals of given size i.e. windowSize.

Reference: Tripwire Histogram API

https://docs.nvidia.com/moj/emdx/tripwire.html#retrieve-tripwire-counts


You can query this curl command to call the Analytics microservice `/metrics/tripwire/histogram` endpoint to get the histogram counts for a tripwire for a given sensor, timerange and window size as seen below. Note the need to specify sensor ID, tripwire ID, fromTimestamp, toTimestamp, objectType and bucketSizeInSec in the parameters

In [ ]:
!curl "http://{jetson_device_ip}:30080/emdx/api/v2/metrics/tripwire/histogram?fromTimestamp={fromTimestamp}&toTimestamp={toTimestamp}&sensorId={sensorId}&tripwireId={tripwireId}&objectType={objectType}&bucketSizeInSec={windowSize}"

In [ ]:
def get_tripwire_histogram_by_direction(endpoint, sensorId, fromTimestamp, toTimestamp, tripwireId, objectType="car,bus,truck", windowSize=5):
    """
    Gets the tripwire histogram and time ranges for a tripwire using API for crossing from each direction for each window of given size

    Keyword arguments:
    endpoint -- the API endpoint of eMDX
    sensorId -- the ID of the camera sensor
    fromTimestamp -- start time for the required data timerange in iso8601 format in milliseconds
    toTimestamp -- end time for the required data timerange in iso8601 format in milliseconds
    tripwireId -- the ID of the tripwire
    windowSize -- the entire duration is divided into windows of given window_size in milliseconds
    objectType -- comma separated list of object classes e.g. car,bus,truck

    Returns a dictionary in the following format:
    {`tripwireId-1`: {"entry": `crossing counts`, "exit": `crossing counts`}, `tripwireId-2`:...}
    """
    parameters = {}
    parameters['sensorId'] = sensorId
    parameters['bucketSizeInSec'] = windowSize
    # This is the histogram bin size
    parameters['toTimestamp'] = toTimestamp
    parameters['fromTimestamp'] = fromTimestamp
    parameters['objectType'] = objectType
    # histogram counts and returned in this timerange

    parameters['tripwireId'] = tripwireId
    encoded_parameters = parse.urlencode(parameters)
    # this API gets histograms of tripwire count in each direction for the given timerange
    url = '{}/api/v2/metrics/tripwire/histogram?{}'.format(endpoint, encoded_parameters)
    #print(f"url : {url}")
    r = requests.get(url,verify=False)
    if r.status_code == 200:
        response = r.json()
    
    time_range = []
    object_counts = {}
    if response:
        tripwires = response['tripwires']
        for tripwire in tripwires:
            if tripwire["id"] == tripwireId:
                histograms = tripwire["histogram"]
                for record in histograms:
                    start = record["start"]
                    end = record["end"]
                    time_range.append(f"{start[:-5]} - {end[:-5]}")
                    events = record["events"]
                    for event in events:
                        direction = event["type"]
                        #print(f"direction : {direction}, object_counts: {object_counts}")
                        if direction not in object_counts:
                            object_counts[direction] = {}
                        objects = event["objects"]
                        for obj in objects:
                            obj_type = obj["type"]
                            if obj_type!="*":
                                if obj_type not in object_counts[direction]:
                                    object_counts[direction][obj_type] = []
                                object_counts[direction][obj_type].append(obj["count"])
                break
    
    return object_counts, time_range
            

#### 2.3.2 Visualize vehicle traffic trends using Stacked bar chart

What is a Stacked Bar Chart?
The stacked bar chart (aka stacked bar graph) extends the standard bar chart from looking at numeric values across one categorical variable to multiple.

This chart represents the vehicle count within a specified timerange (as specified by fromTimestamp/toTimstamp parameters) for each direction. The timerange is divided into fixed windows (as specified by windowSize parameter). Within each time window, the chart uses color-coded bars to denote the count of each vehicle type i.e. car, bus, truck.
The height of each color within a bar directly corresponds to vehicle count for that category.

To create a stacked bar chart:

First, we will define a function that invokes the histogram endpoint.
This function returns the counts of each objects in the scene for each window within the given time slot and also the time ranges for each window and all direction i.e. entry,exit
We then use these outputs to plot a separate stacked bar chart for each direction.

Note: Graph is only plotted if there in any non-zero count for any object for a particular direction 

In [ ]:
for idx, tripwireId in enumerate(tripwireIds):
    object_histogram_by_direction, time_range_list = get_tripwire_histogram_by_direction(endpoint, sensorId, fromTimestamp, toTimestamp, tripwireId, objectType, windowSize)

    # plot the histogram for each direction
    for direction in object_histogram_by_direction:
        tripwireName = tripwireIdMap[tripwireId]
        print("==========================================================")
        print(f"**Histogram for tripwire - {tripwireName} , direction={direction}**")
        print("==========================================================")
        if object_histogram_by_direction[direction] != {}:
            plot_stacked_bar_chart(time_range_list, object_histogram_by_direction[direction], fromTimestamp, toTimestamp)

#### 2.3.3 Retrieve the Tripwire entry / exits counts for a given time range from API
This function calls Tripwire Analytics Count API to get entry and exit counts for a given timerange for given object types



You can query this curl command to call the Analytics microservice /metrics/tripwire endpoint to get the counts for a tripwire for a given sensor, timerange for all directions as seen below. Note the need to specify sensor ID, tripwire ID, fromTimestamp, toTimestamp, objectType in the parameters

In [ ]:
!curl "http://{jetson_device_ip}:30080/emdx/api/v2/metrics/tripwire?fromTimestamp={fromTimestamp}&toTimestamp={toTimestamp}&sensorId={sensorId}&tripwireId={tripwireId}&objectType={objectType}"

In [ ]:
def get_tripwire_counts_by_direction(endpoint, sensorId, fromTimestamp, toTimestamp, tripwireId, objectType="car,bus,truck"):
    """
    Gets the tripwire crossing counts for a tripwire using API for crossing from each direction

    Keyword arguments:
    endpoint -- the API endpoint of eMDX
    sensorId -- the ID of the camera sensor
    fromTimestamp -- start time for the required data timerange in iso8601 format in milliseconds
    toTimestamp -- end time for the required data timerange in iso8601 format in milliseconds
    tripwireId -- the ID of the tripwire
    objectType -- comma separated list of object classes e.g. car,bus,truck

    Returns a dictionary in the following format:
    {`tripwireId-1`: {"entry": `crossing counts`, "exit": `crossing counts`}, `tripwireId-2`:...}
    """
    parameters = {}
    parameters['sensorId'] = sensorId
    # This is the histogram bin size
    parameters['toTimestamp'] = toTimestamp
    parameters['fromTimestamp'] = fromTimestamp
    parameters['objectType'] = objectType
    parameters['tripwireId'] = tripwireId


    # counts w.r.t each object type in this timerange for each direction

    total_object_counts = {}
    encoded_parameters = parse.urlencode(parameters)
    # this API gets histograms of tripwire count in each direction for the given timerange
    url = '{}/api/v2/metrics/tripwire?{}'.format(endpoint, encoded_parameters)
    r = requests.get(url,verify=False)
    if r.status_code == 200:
        response = r.json()
    
    result_dict = {}
    
    if response:
        tripwirekpis = response["tripwireKpis"]
        for tripwirekpi in tripwirekpis:
            if tripwirekpi["id"] == tripwireId:
                events = tripwirekpi["events"]
                for event in events:
                    direction = event["type"]
                    if direction not in result_dict:
                        result_dict[direction] = {}
                    objects = event["objects"]
                    for obj in objects:
                        obj_type = obj["type"]
                        if obj_type !="*":
                            result_dict[direction][obj_type] = obj["count"]
                break
        

    return result_dict

get_tripwire_counts_by_direction(endpoint, sensorId, fromTimestamp, toTimestamp, tripwireIds[0], objectType)

In [ ]:
all_tripwire_results = {}
for tripwireId in tripwireIds:
    all_tripwire_results[tripwireId] = get_tripwire_counts_by_direction(endpoint, sensorId, fromTimestamp, toTimestamp, tripwireId, objectType)
all_tripwire_results

#### 2.3.4 Get Image snapshot of the live stream using VST API

We will now retrieve the image snapshot of the live stream from VST. This is required to visualize the traffic distribution across groups and visualize it on the scene image snapshot.

- Retrieve the stream ID of the specified stream by name using method get_stream_id_from_sensor_id
- Retrieve the image snapshot for a given stream ID by using method get_snapshot_data
- Store it as an OpenCV image object

Reference: VST API Guide

https://docs.nvidia.com/moj/vst_API/index.html

In [ ]:
def get_stream_id_from_sensor_id(vst_endpoint, sensorId):
  """
  Gets the stream ID of the camera sensor from VST API

  Keyword arguments:
  vst_endpoint -- the API endpoint of VST
  sensorId -- the ID of the camera sensor

  Return Value:
  streamID -- stream ID of the camera sensor
  """
  # API to get all the live camera streams
  url = '{}/api/v1/live/streams'.format(vst_endpoint)
  r = requests.get(url,verify=False)
  if r.status_code == 200:
    all_streams_response = r.json()
    for streams_data in all_streams_response:
      for _, streams_list in streams_data.items():
        for stream in streams_list:
          if stream["name"] == sensorId:
            return stream["streamId"]
          
def get_snapshot_data(vst_endpoint, streamId):
  """
  Gets the snapshot data of the camera sensor from VST API

  Keyword arguments:
  vst_endpoint -- the API endpoint of VST
  streamId -- the stream ID of the camera sensor

  Returns API response with image content in bytes
  """
  # API to get sensor snapshot
  url = '{}/api/v1/live/stream/{}/picture'.format(vst_endpoint, streamId)
  r = requests.get(url,verify=False)
  if r.status_code == 200:
    return r.content

You can query this curl command to call the VST microservice `v1/live/streams` endpoint to get all the current live streams. This contains the streamId necessary to make the call to fetch snapshot data.

In [ ]:
!curl http://{jetson_device_ip}:30080/vst/api/v1/live/streams

You can query this cURL command to call VST microservice `/v1/live/stream/{streamId}/picture` endpoint to get the snapshot of the stream with a streamID. Note the need to specify stream ID which can be fetched from the `/v1/live/streams` endpoint.

`curl http://{jetson_device_ip}:30080/vst/api/v1/live/stream/{vst_streamId}/picture`

#### 2.3.5 Visualize Tripwire Group Counts on the scene image snapshot

In [ ]:
def overlay_tripwire_analytics(cv2_image, tripwire_config, tripwire_map, tripwire_count_results, tripwire_color):
    """
    Plots all the tripwire analytics results on the image. It also tells the distribution percentage of counts between different tripwires

    Keyword arguments:
    cv2_image -- an opencv image object which is a snapshot of the camera sensor scene
    tripwire_config -- tripwire config response returned by the eMDX API endpoint
    tripwire_map -- the dictionary that maps tripwire ID to tripwire name
    tripwire_count_results -- dictionary which contains tripwire counts for which this function will overlay the analytics
    tripwire_color -- color of the tripwire to be overlayed on the image
    """  
    tripwires = tripwire_config['tripwires']
    height, width = cv2_image.shape[:2]
    arrow_head = (width//2, height//2)

    total_crossings = 0
    
    for tripwire in tripwires:
        analytics_overlay_details = []
        tripwireId = tripwire['id']
        if tripwireId in tripwire_count_results:
            coordinates = np.array([(point['x'], point['y']) for point in tripwire['wire']])
            # draws the tripwire
            cv2.polylines(cv2_image, [coordinates.astype(int)], isClosed=False, color=tripwire_color, thickness=2)
            # writes the tripwire name
            cv2.putText(cv2_image, str(tripwire_map[tripwire['id']]),\
                     (int(coordinates[0][0] + 5), int(coordinates[0][1] - 5)), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=tripwire_color, thickness=2)
            crossing_directions = list(tripwire_count_results[tripwireId].keys())
            text_pos = (coordinates[1] + coordinates[2])//2
            
            for direction in crossing_directions:
                if any(tripwire_count_results[tripwireId][direction].values()):
                    analytics_overlay_details.append({"text_pos": text_pos, "obj_count": tripwire_count_results[tripwireId][direction], "direction": direction})
                    total_crossings+=sum(tripwire_count_results[tripwireId][direction].values())
    
            text_pos = (coordinates[0][0]+ 20, coordinates[0][1]+ 35)
            for analytics in analytics_overlay_details:
                #text_pos = (int(analytics["text_pos"][0]), int(analytics["text_pos"][1] - 20))
                text = f"{analytics['direction']}: " 
                text += ', '.join([f"{key}={value}" for key,value in analytics["obj_count"].items()])
                # overlays the counts of each object for each direction
                cv2.putText(cv2_image, text,\
                            text_pos, cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=tripwire_color, thickness=2)
                text_pos = (text_pos[0], text_pos[1]+ 40)

    cv2.destroyAllWindows()

    return cv2_image

Utilize the above function to create an overlay for all the tripwire crossing analytics. The number shown next to each tripwire specifies the number of crossings.

In [ ]:
streamId = get_stream_id_from_sensor_id(vst_endpoint, sensorId)
img_content = get_snapshot_data(vst_endpoint, streamId)
cv2_image = get_vst_snapshot(img_content)
# First overly the group-1 statistic 
img = overlay_tripwire_analytics(cv2_image, tripwire_config, tripwireIdMap, all_tripwire_results, (255, 255, 0))

# Save the image
cv2.imwrite("tripwire-viz.jpg", img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.rcParams['figure.figsize'] = [20, 10]
plt.imshow(img)
plt.show()

## 3. Heatmap based Traffic Movement Analytics
### What is a heatmap ?
A heatmap is a visual representation of time spent by an object in a particular location. In this section of the notebook, we analyze vehicle trajectories using heatmaps.

Firstly the trajectories captured from a sensor are obtained for a given time window. Further, the trajectories are then classified based on the tripwires they cross. Then, a 2D histogram of required size is computed for points in all the trajectories of a class. The 2D histogram is further smoothened to obtain the final heatmap

To create a heatmap:
* First, we will define a function that invokes the behavior endpoint
  * This function returns the behaviours of all the objects in the scene within the given time slot
* We then convert the response from behavior endpoint, into trajectories based on their object ID
* Then we define a set of functions that are needed to classify the trajectories using the tripwires

### 3.1 Get object trajectories within a given time window
Trajectory is a list of (x, y) coordinates of mid point of lower edge of the detected object bounding boxes.
In order to build a heatmap , we use these points in the Trajectory.


#### 3.1.1 Retrieve behaviors from API

We can use `/emdx/api/v2/behavior` endpoint to get the list of all object trajectory points sorted by their timestamp within a given time window.
Each entry in the response of the endpoint includes the following:
* timestamp
* Object ID
* X, Y coordinates of the trajectory point of the object

The `get_behaviors` method below invokes the endpoint with sensorId, time window specified by their to and from timestamps.

In [ ]:
def get_behaviors(endpoint, sensorId, fromTimestamp, toTimestamp, objectType):
    """
    This method calls the /emdx/api/v2/behavior endpoint
    
    Keyword arguments:
    sensorId -- ID of the sensor as per the config
    fromTimestamp -- a datetime object representing the start of time window
    toTimestamp -- a datetime object representing the end of time window
    """
    fromTimestamp = fromTimestamp.isoformat(timespec='milliseconds').replace('+00:00', 'Z')
    toTimestamp = toTimestamp.isoformat(timespec='milliseconds').replace('+00:00', 'Z')
    objectType = objectType
    
    parameters = {}
    parameters['sensorId'] = sensorId
    # This is the histogram bin size
    parameters['toTimestamp'] = toTimestamp
    parameters['fromTimestamp'] = fromTimestamp
    parameters['objectType'] = objectType
    # histogram counts and returned in this timerange

    encoded_parameters = parse.urlencode(parameters)
    # this API gets histograms of tripwire count in each direction for the given timerange
    url = '{}/api/v2/behavior?{}'.format(endpoint, encoded_parameters)
    
    #url = f"{endpoint}/api/v2/behavior?sensorId={sensorId}&fromTimestamp={fromTimestamp}&toTimestamp={toTimestamp}&objectType={objectType}"

    payload = ""
    headers = {}
    behaviors = []
    try:
        r = requests.request("GET", url, headers=headers, data=payload)
        if r.status_code == 200:
            response = r.json()
            #response = requests.request("GET", url, headers=headers, data=payload)
            behaviors = response['behaviors']
    except:
        print(f"Error in getting trajectories for sensor {sensorId} from {fromTimestamp} to {toTimestamp}")
        print(url)
        print(response.text)
    
    return behaviors

#### 3.1.2 Extract trajectories from behaviors

The maximum allowed time window for the `/behavior` endpoint is 5 seconds. Therefore, it needs to be called multiple times in order to obtain the behaviors for time window larger than the specified limit.

Therefore, the `get_trajectories` method given below aggregates the object behaviors for each 5 second interval.
It then creates and returns a list of dataframes, where each dataframe contains the (timestamp, x coordinate, y coordinate) per object ID.

In [ ]:
def get_trajectories(endpoint, sensorId, fromTimestamp, toTimestamp, objectType):
    """
    This method splits the input time window into 5 second intervals.
    On each 5 second interval, it calls the get_behaviors method
    It accumulates and returns all the trajectories collected from each interval
    """
    print(f"Getting trajectories for sensor {sensorId} from: {fromTimestamp}, to: {toTimestamp}")
    trajs = []
    objects = {}
    
    while fromTimestamp < toTimestamp:
        delta_in_secs = (toTimestamp - fromTimestamp).seconds
        min_delta_in_secs = min(5, delta_in_secs)
        
        start_time = fromTimestamp
        end_time = fromTimestamp + timedelta(seconds=min_delta_in_secs)
        
        fromTimestamp = end_time
        behaviors = get_behaviors(endpoint, sensorId, start_time, end_time, objectType)

        for behavior in behaviors:
            obj_id = behavior["object"]["id"]
            obj_type = behavior["object"]["type"]
            obj_coords = behavior['locations']['coordinates']

            points = [
                [t, x, y]
                for t, [x, y] in obj_coords
            ]

            obj_points = objects.get(obj_id, [])
            obj_points += points
            objects[obj_id] = obj_points
    
    for points in objects.values():
        df = pd.DataFrame(points)
        df.columns = ['ts', 'x', 'y']

        trajs.append(df)

    print(f"Got {len(trajs)} trajectories")
    
    return trajs

The code below, invokes the `get_trajectories` method for a time window that lasts 1 minute from the current time (as defined by `time_delta_in_mins` variable). This may take a few seconds to run.

*However, to account for the latency in computing the behaviors and to allow for any detections to happen within the given time window, we offset the current timestamp by 5 minutes.*

In [ ]:
currentTimestamp = datetime.now(timezone.utc) - timedelta(minutes=5)
time_delta_in_mins = 1

fromTimestamp = currentTimestamp - timedelta(minutes=time_delta_in_mins)
toTimestamp = currentTimestamp

trajs = get_trajectories(endpoint, sensorId, fromTimestamp, toTimestamp, objectType)


Before proceeding to computing and plotting the heatmap, we grab an image frame from the video stream of sensor using VST endpoint.

We use this image frame to determine the dimensions of the heatmap as well as to use it as the background for overlay

In [ ]:
# Grab the latest camera frame from the sensor using VST endpoint
streamId = get_stream_id_from_sensor_id(vst_endpoint, sensorId)
img_content = get_snapshot_data(vst_endpoint, streamId)
image = get_vst_snapshot(img_content)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

img_width, img_height = image.shape[1], image.shape[0]

### 3.2 Generate Heatmap of Vehicle Movement

This section defines the functions to compute and plot the heatmap of a list of trajectories

Try different values for scale and smoothness of the generated heatmap
* scale - controls the resolution of output heatmap (preferred values are from 2 to 10)
  * higher value of scale, results in lower resolution of heatmap
* smoothness - controls the variance of smoothening (gaussian) filter (preferred values are from 2 to 10)
  * higher value of smoothness creates highly smoothened heat map

In [ ]:
from utils import plot_heatmap
H = plot_heatmap(trajs, image, name='Heatmap of all trajectories')